# 1. Loaded SQLite and Database
- 'CourseData.db'

In [37]:
%reload_ext sql
%sql sqlite:///CourseData.db

'Connected: @CourseData.db'

In [38]:
%%sql
DROP TABLE IF EXISTS CourseData;

 * sqlite:///CourseData.db
Done.


[]

## Drop tables to start from scratch

In [60]:
%%sql
sqlite:///CourseData.db
DROP TABLE IF EXISTS COURSECATALOGS;
DROP TABLE IF EXISTS COURSES;
DROP TABLE IF EXISTS COURSEMEETINGS;

Done.
Done.
Done.


[]

# 2. COUNT 3 .csv Files
- ensured values returned are correct

In [61]:
%%sql
SELECT Count(*) FROM import_course_meetings;

 * sqlite:///CourseData.db
Done.


Count(*)
317339


In [62]:
%%sql
SELECT Count(*) FROM import_courses;

 * sqlite:///CourseData.db
Done.


Count(*)
15955


In [63]:
%%sql
SELECT Count(*) FROM import_Course_Catalog;

 * sqlite:///CourseData.db
Done.


Count(*)
4441


# 3. CREATE Tables


In [64]:
%%sql
CREATE TABLE COURSECATALOGS (
    CCID INTEGER PRIMARY KEY,
    catalog_id TEXT,
    program_code TEXT,    
    program_name TEXT,
    course_title TEXT,
    prereqs TEXT,    
    coreqs TEXT,
    fees TEXT,
    attributes TEXT,    
    description TEXT
);

 * sqlite:///CourseData.db
Done.


[]

In [67]:
%%sql
CREATE TABLE COURSES (
                 term TEXT,
                 crn TEXT,
                 catalog_id TEXT,
                 section TEXT,
                 credits TEXT,
                 title TEXT,
                 meetings TEXT,
                 timecodes TEXT,
                 primary_instructor TEXT,
                 cap TEXT,
                 act TEXT,
                 rem TEXT,
                 PRIMARY KEY (crn,term)
    );


 * sqlite:///CourseData.db
Done.


[]

In [66]:
%%sql
CREATE TABLE COURSEMEETINGS (
    CMID INTEGER PRIMARY KEY,
    term TEXT,
    crn TEXT,
    location TEXT NOT NULL,	
    day TEXT NOT NULL,
    start TEXT NOT NULL,
    end TEXT NOT NULL
);

 * sqlite:///CourseData.db
Done.


[]

# 4. Do INSERT Statements

In [46]:
%%sql
DELETE FROM COURSECATALOGS;

INSERT INTO COURSECATALOGS (catalog_id, program_code, program_name, course_title, prereqs, coreqs,fees,attributes,description) 
SELECT DISTINCT catalog_id, program_code, program_name, course_title, prereqs, coreqs, fees, attributes, description
    FROM import_Course_Catalog;

 * sqlite:///CourseData.db
0 rows affected.
2221 rows affected.


[]

In [47]:
%%sql
SELECT * FROM COURSECATALOGS
limit 1;

 * sqlite:///CourseData.db
Done.


CCID,catalog_id,program_code,program_name,course_title,prereqs,coreqs,fees,attributes,description
1,AN 0301,AN,Asian Studies,Independent Study,,,,,Students undertake an individualized program of study in consultation with a director from the Asian studies faculty.


In [48]:
%%sql 
DELETE FROM COURSEMEETINGS;

INSERT INTO COURSEMEETINGS (term, crn, location, day, start, end)
SELECT DISTINCT term, crn, location, day, start, end
    FROM import_course_meetings
;

 * sqlite:///CourseData.db
0 rows affected.
311142 rows affected.


[]

In [49]:
%%sql
SELECT * FROM COURSEMEETINGS
LIMIT 2;

 * sqlite:///CourseData.db
Done.


CMID,term,crn,location,day,start,end
1,Fall2014,70384,DSB 105,T,2014-09-02T08:00:00,2014-09-02T09:15:00
2,Fall2014,70384,DSB 105,F,2014-09-05T08:00:00,2014-09-05T09:15:00


In [50]:
%%sql
INSERT INTO COURSES (term, crn, catalog_id, section, credits, title, meetings, timecodes, cap, act, rem)
SELECT DISTINCT term, crn, catalog_id, section, credits, title, meetings, timecodes, cap, act, rem
                FROM import_courses

 * sqlite:///CourseData.db
15938 rows affected.


[]

In [51]:
%%sql
SELECT * FROM COURSES
limit 5;

 * sqlite:///CourseData.db
Done.


term,crn,catalog_id,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem
Fall2014,70384,AC 0011,C01,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0800am-0915am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0800am-0915am 09/02-12/08 DSB 105'],None,0,31,-31
Fall2014,70385,AC 0011,C02,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '0930am-1045am', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 0930am-1045am 09/02-12/08 DSB 105'],None,0,31,-31
Fall2014,70382,AC 0011,C03,3.000,Introduction to Financial Accounting,"[{'days': 'TF', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 105'}]",['TF 1230pm-0145pm 09/02-12/08 DSB 105'],None,0,31,-31
Fall2014,70291,AC 0011,C04,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '1100am-1215pm', 'dates': '09/02-12/08', 'location': 'DSB 111'}]",['MR 1100am-1215pm 09/02-12/08 DSB 111'],None,0,29,-29
Fall2014,70350,AC 0011,C05,3.000,Introduction to Financial Accounting,"[{'days': 'MR', 'times': '1230pm-0145pm', 'dates': '09/02-12/08', 'location': 'DSB 111'}]",['MR 1230pm-0145pm 09/02-12/08 DSB 111'],None,0,30,-30


# 5. Run SELECT Queries

In [52]:
%reload_ext sql
%sql sqlite:///CourseData.db

'Connected: @CourseData.db'

## SELECT Query for all unique classrooms 
- only included those with a character lenght of 7 (some had 2, 8, etc.)
- can't figure out how to get a distinct location when including additinonal fields.

In [53]:
%%sql
SELECT DISTINCT 
location
FROM COURSEMEETINGS
WHERE LENGTH(location) = 7
ORDER BY location
LIMIT 2
;

 * sqlite:///CourseData.db
Done.


location
BCC 200
BLM 112


In [68]:
%%sql
SELECT COURSES.catalog_id, COURSEMEETINGS.CMID FROM COURSES
JOIN COURSEMEETINGS ON (COURSEMEETINGS.term=import_courses.term AND COURSEMEETINGS.crn=import_courses.crn)
limit 5;

 * sqlite:///CourseData.db
(sqlite3.OperationalError) no such column: import_courses.term [SQL: 'SELECT COURSES.catalog_id, COURSEMEETINGS.CMID FROM COURSES\nJOIN COURSEMEETINGS ON (COURSEMEETINGS.term=import_courses.term AND COURSEMEETINGS.crn=import_courses.crn)\nlimit 5;'] (Background on this error at: http://sqlalche.me/e/e3q8)


## SELECT Query displaying all courses in MSBA Program '18-'19
- still have to get rid of blank by making defining description as NOT NULL.
- is there an easier way?

In [69]:
%%sql
SELECT course_title AS Course, program_name AS Program, catalog_id AS Code, description AS Description
FROM COURSECATALOGS
WHERE program_name = 'Information Systems'
AND
Code LIKE "IS 05%"
ORDER BY program_name
LIMIT 2;

 * sqlite:///CourseData.db
Done.


Course,Program,Code,Description


# JOIN Testing

In [24]:
%%sql
SELECT *
FROM COURSES
LIMIT 10;

 * sqlite:///CourseData.db
Done.


CID,catalog_id,crn,term,section,credits,title,meetings,timecodes,primary_instructor,cap,act,rem,CCID,TID,CMID


In [25]:
%%sql
SELECT t.primary_instructor, c.catalog_id
FROM COURSES as c
JOIN TEACHERS as t ON c.TID = t.TI;

 * sqlite:///CourseData.db
(sqlite3.OperationalError) no such column: t.TI [SQL: 'SELECT t.primary_instructor, c.catalog_id\nFROM COURSES as c\nJOIN TEACHERS as t ON c.TID = t.TI;'] (Background on this error at: http://sqlalche.me/e/e3q8)


In [26]:
%%sql
SELECT TID, primary_instructor FROM TEACHERS
limit 10;


 * sqlite:///CourseData.db
Done.


TID,primary_instructor
1,Michael P. Coyne
2,Rebecca I. Bloch
3,Paul Caster
4,Jo Ann Drusbosky
5,Arleen N. Kardos
6,Scott M Brenner
7,Kevin C. Cassidy
8,Bruce Bradford
9,Milo W. Peck
10,Stephen E. Yost


Have to finish insterting COURSES column

In [27]:
%%sql
VACUUM;

 * sqlite:///CourseData.db
Done.


[]